# Candlestick pattern modeling

### Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [2]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [3]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-6)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [4]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

### Load Price Data

In [5]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
len(ticker_data)

6461

In [6]:
import candlestick.price_action as pa

In [7]:
data = ticker_data.copy()

In [8]:
# BACK_BARS=250
# data = data.tail(BACK_BARS)
data = data[data.index >  '2024-01-01 08:00:00']

In [9]:
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma_1w'] = data['above_ma'].rolling(250).sum()
data['total_below_ma_1w'] = data['below_ma'].rolling(250).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else ('up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)
data["prev_Volume"] = data["Volume"].shift(1)

In [10]:
%%time
data = pa.pattern_modeling(data, 'reversal')

CPU times: user 4.48 s, sys: 28.4 ms, total: 4.51 s
Wall time: 4.48 s


In [11]:
# data[data.model.str.contains("bearish")]

In [12]:
this_month = data[data.index >  '2024-09-01 08:00:00']
this_month['oc_dif_5bars'] = this_month['oc_dif'].rolling(5).sum().round(2)
this_month["prev_oc_dif_5bars"] = this_month["oc_dif_5bars"].shift(1)

In [13]:
def has_bullish_pattern(model):
    if "bullish" in model or "rising" in model:
        return True
    return False

def has_bearish_pattern(model):
    if "bearish" in model or "falling" in model:
        return True
    return False
    
def cal_signal(r):
    signal = ''
    if r['Volume'] < r['prev_Volume']:
        return signal
    if has_bullish_pattern(r['model']):
        if r['trend'] == 'up' and r['prev_oc_dif_5bars'] < 0:
            signal = 'long'
        else:
            signal = 'close short(*)'
    elif has_bearish_pattern(r['model']):
        if r['trend'] == 'down' and r['prev_oc_dif_5bars'] > 0:
            signal = 'short'
        else:
            signal = 'close long(*)'
    return signal

In [14]:
this_month['signal'] = this_month.apply(lambda r: cal_signal(r), axis=1)

In [15]:
this_month[(this_month.index >  '2024-08-21 08:00:00') & (this_month.model != '')]

,Open,High,Low,Close,Volume,ma_line,above_ma,below_ma,total_above_ma_1w,total_below_ma_1w,...,upper_wick,tail,oc_dif,body,color,candlestick,model,oc_dif_5bars,prev_oc_dif_5bars,signal
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-04 09:25:00,1312.2,1313.3,1312.1,1313.0,2813,1327.355,0,1,159.0,91.0,...,0.3,0.1,0.8,0.8,white,,bullish_piercing,-4.4,-8.0,
2024-09-04 09:40:00,1314.0,1314.0,1313.3,1313.3,1886,1324.360,0,1,156.0,94.0,...,0.0,0.0,-0.7,0.7,black,marubozu,bearish_engulfing,-0.1,2.2,
2024-09-04 09:45:00,1313.3,1314.5,1313.1,1314.4,2253,1323.385,0,1,155.0,95.0,...,0.1,0.2,1.1,1.1,white,,bullish_engulfing,2.3,-0.1,long
2024-09-04 10:00:00,1315.0,1315.0,1314.1,1314.5,2355,1320.555,0,1,154.0,96.0,...,0.0,0.4,-0.5,0.5,black,,bearish_dark_cloud_cover,0.5,1.1,
2024-09-04 10:15:00,1314.0,1314.4,1313.6,1313.8,900,1317.640,0,1,151.0,99.0,...,0.4,0.2,-0.2,0.2,black,,bearish_dark_cloud_cover,-0.8,-0.3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-24 10:15:00,1351.9,1352.2,1351.4,1352.0,3218,1352.765,0,1,112.0,138.0,...,0.2,0.5,0.1,0.1,white,,bullish_meeting_line,-0.4,-1.1,close short(*)
2024-10-24 10:30:00,1352.2,1352.3,1351.2,1351.3,1938,1352.585,0,1,112.0,138.0,...,0.1,0.1,-0.9,0.9,black,,bearish_engulfing,-1.0,-0.3,close long(*)
2024-10-24 10:50:00,1349.7,1351.0,1348.8,1350.8,4868,1352.150,0,1,112.0,138.0,...,0.2,0.9,1.1,1.1,white,,bullish_engulfing,-1.4,-2.3,close short(*)


In [16]:
# Today
data.tail(50)

,Open,High,Low,Close,Volume,ma_line,above_ma,below_ma,total_above_ma_1w,total_below_ma_1w,...,prev_Volume,min_OC,max_OC,upper_wick,tail,oc_dif,body,color,candlestick,model
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-23 13:25:00,1352.4,1352.6,1351.5,1351.7,2752,1349.635,1,0,102.0,148.0,...,4981.0,1351.7,1352.4,0.2,0.2,-0.7,0.7,black,,bearish_engulfing
2024-10-23 13:30:00,1351.6,1351.6,1350.2,1350.7,5248,1349.745,1,0,103.0,147.0,...,2752.0,1350.7,1351.6,0.0,0.5,-0.9,0.9,black,,
2024-10-23 13:35:00,1350.7,1352.3,1350.4,1352.0,3043,1349.995,1,0,104.0,146.0,...,5248.0,1350.7,1352.0,0.3,0.3,1.3,1.3,white,,bullish_engulfing
2024-10-23 13:40:00,1352.0,1353.0,1351.8,1352.5,4853,1350.280,1,0,105.0,145.0,...,3043.0,1352.0,1352.5,0.5,0.2,0.5,0.5,white,,
2024-10-23 13:45:00,1352.6,1353.9,1351.9,1353.9,5556,1350.600,1,0,106.0,144.0,...,4853.0,1352.6,1353.9,0.0,0.7,1.3,1.3,white,,
2024-10-23 13:50:00,1353.8,1353.9,1350.8,1351.3,6363,1350.780,1,0,107.0,143.0,...,5556.0,1351.3,1353.8,0.1,0.5,-2.5,2.5,black,,bearish_tasuki_line
2024-10-23 13:55:00,1351.1,1351.8,1350.7,1351.6,4866,1351.005,1,0,108.0,142.0,...,6363.0,1351.1,1351.6,0.2,0.4,0.5,0.5,white,,
2024-10-23 14:00:00,1351.8,1351.9,1350.5,1350.5,3458,1351.110,0,1,107.0,143.0,...,4866.0,1350.5,1351.8,0.1,0.0,-1.3,1.3,black,,bearish_engulfing
2024-10-23 14:05:00,1350.6,1351.5,1348.3,1348.9,7444,1351.120,0,1,106.0,144.0,...,3458.0,1348.9,1350.6,0.9,0.6,-1.7,1.7,black,,
